In [1]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score
import statsmodels.api as sm
import warnings

warnings.simplefilter(action='ignore', category=Warning)
from sklearn.linear_model import HuberRegressor
# pd.set_option('display.max_rows', None)
# more

In [2]:
df = pd.read_csv('data/factors_1970.csv', parse_dates=['DATE'])
# df = pd.read_csv('data/features_subset.csv', parse_dates=['DATE'])
# df = pd.read_csv('factors_1900.csv', parse_dates=['DATE'])
df.drop(columns=['sic2'], inplace=True)

In [3]:
#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,macro_mkt-rf,macro_hml,macro_smb
0,10401,1970-02-27,26227356.0,0.253755,0.064391,-0.057929,13.348085,0.019852,-0.112202,-0.017995,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
1,10604,1970-02-27,3196008.0,0.743947,0.553457,-0.228256,13.348085,0.044046,-0.252877,-0.164006,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
2,10786,1970-02-27,1133566.5,0.608122,0.369813,-0.046833,12.456023,0.025765,-0.302840,-0.050691,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
3,10890,1970-02-27,2662344.0,0.833271,0.694341,0.001993,13.348085,0.035138,-0.127664,-0.018268,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58
4,11260,1970-02-27,1342376.0,1.059626,1.122808,0.034205,13.348085,0.032067,-0.349402,-0.185455,...,2.757288,0.670147,0.035855,0.0713,-0.0126,0.0085,0.001059,5.13,3.93,-2.58


In [4]:
df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [21]:
p=0.3 
df_top= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_bottom = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


In [6]:

def calculate_r2(y_true, y_pred, in_sample=True, benchmark=None):
    if in_sample:
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - np.mean(y_true)) ** 2))
    else:
        if benchmark is None:
            raise ValueError("Benchmark must be provided for out-of-sample R-squared calculation.")
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - benchmark) ** 2))

In [7]:
features = df.columns[~df.columns.isin(['DATE', 'DATE2', "mvel2",'permno',"permno2",'risk_premium'])].tolist()
df[features]=df.groupby('DATE')[features].rank(pct=True)

df[features] = 2*df[features] - 1

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                #    test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df[features]
y = df[['risk_premium']]

predictions = []
y_val_list = []
dates = []
dic_r2_all = {}


for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):


    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    huber = HuberRegressor(max_iter=1000)

    # fitting the huber model
    huber.fit(X_train, y_train)

    # predicting on the training set and assiging to the training list
    preds_train = huber.predict(X_train)

    preds_val = huber.predict(X_val) 
    predictions.append(preds_val)
    dates.append(y_val.index)
    y_val_list.append(y_val)
    
    #Calculate OOS model performance the for current window
    r2_train = 1-np.sum(pow(y_train['risk_premium']-preds_train,2))/np.sum(pow(y_train['risk_premium'],2))
    r2_val = 1-np.sum(pow(y_val['risk_premium']-preds_val,2))/np.sum(pow(y_val['risk_premium'],2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all["r2." + str(y_val['risk_premium'].index)] = r2_val
    

    print(f'R2 {y_train.index[0][0].date()} - {y_train.index[-1][0].date()} training set {r2_train}')
    print(f'R2 {y_val.index[0][0].date()} - {y_val.index[-1][0].date()} validation set {r2_val}')
    print('---')

predictions_all_full = np.concatenate(predictions, axis=0)
y_test_list_all_full = np.concatenate(y_val_list, axis=0)
dates_all_full = np.concatenate(dates, axis=0)

# R2FULL = 1-np.sum(pow(y_test_list_all_full-predictions_all_full,2))/np.sum(pow(y_test_list_all_full,2))
# print("R2OOS Linear Regression: ", R2FULL)
R2FULL = r2_score(y_test_list_all_full, predictions_all_full)
print("R2OOS Linear Regression: ", R2FULL)

Train period: 1980-01-31 - 1985-01-31 ,val period: 1985-01-31 - 1987-01-31 # train records 18589 ,# val records 11286
Train period: 1981-01-31 - 1986-01-31 ,val period: 1986-01-31 - 1988-01-31 # train records 20125 ,# val records 13057
Train period: 1982-01-31 - 1987-01-31 ,val period: 1987-01-31 - 1989-01-31 # train records 22963 ,# val records 12572
Train period: 1983-01-31 - 1988-01-31 ,val period: 1988-01-31 - 1990-01-31 # train records 26711 ,# val records 12781
Train period: 1984-01-31 - 1989-01-31 ,val period: 1989-01-31 - 1991-01-31 # train records 28226 ,# val records 13403
Train period: 1985-01-31 - 1990-01-31 ,val period: 1990-01-31 - 1992-01-31 # train records 30708 ,# val records 13616
Train period: 1986-01-31 - 1991-01-31 ,val period: 1991-01-31 - 1993-01-31 # train records 32391 ,# val records 15806
Train period: 1987-01-31 - 1992-01-31 ,val period: 1992-01-31 - 1994-01-31 # train records 33038 ,# val records 17371
Train period: 1988-01-31 - 1993-01-31 ,val period: 1993-

MemoryError: Unable to allocate 10.3 TiB for an array with shape (1190354, 1190354) and data type float64

In [ ]:

tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df_top[features]
y = df_top[['risk_premium']]

predictions_top = []
y_val_list_top = []
dates_top = []
dic_r2_all_top = {}


for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    huber = HuberRegressor(max_iter=1000)

    # fitting the huber model
    huber.fit(X_train, y_train)

    # predicting on the training set and assiging to the training list
    preds_train = huber.predict(X_train)

    preds_val = huber.predict(X_val) 
    predictions_top.append(preds_val)
    dates_top.append(y_val.index)
    y_val_list_top.append(y_val)
    
    #Calculate OOS model performance the for current window
    r2_train = 1-np.sum(pow(y_train['risk_premium']-preds_train,2))/np.sum(pow(y_train['risk_premium'],2))
    r2_val = 1-np.sum(pow(y_val['risk_premium']-preds_val,2))/np.sum(pow(y_val['risk_premium'],2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all_top["r2." + str(y_val['risk_premium'].index)] = r2_val
    

    print(f'R2  training set {r2_train}')
    print(f'R2  validation set {r2_val}')
    print('---')

predictions_all_full_top = np.concatenate(predictions_top, axis=0)
y_test_list_all_full_top = np.concatenate(y_val_list_top, axis=0)
dates_all_full_top = np.concatenate(dates_top, axis=0)

# R2TOP = 1-np.sum(pow(y_test_list_all_full_top-predictions_all_full_top,2))/np.sum(pow(y_test_list_all_full_top,2))
# print("R2OOS Linear Regression: ", R2TOP)


R2TOP = r2_score(y_test_list_all_full_top, predictions_all_full_top)
print("R2OOS Linear Regression: ", R2TOP)

Train period: 1980-01-31 - 1985-01-31 ,val period: 1985-01-31 - 1987-01-31 # train records 5549 ,# val records 3377
Train period: 1981-01-31 - 1986-01-31 ,val period: 1986-01-31 - 1988-01-31 # train records 6012 ,# val records 3906
Train period: 1982-01-31 - 1987-01-31 ,val period: 1987-01-31 - 1989-01-31 # train records 6863 ,# val records 3764
Train period: 1983-01-31 - 1988-01-31 ,val period: 1988-01-31 - 1990-01-31 # train records 7988 ,# val records 3826
Train period: 1984-01-31 - 1989-01-31 ,val period: 1989-01-31 - 1991-01-31 # train records 8445 ,# val records 4008
Train period: 1985-01-31 - 1990-01-31 ,val period: 1990-01-31 - 1992-01-31 # train records 9190 ,# val records 4072
Train period: 1986-01-31 - 1991-01-31 ,val period: 1991-01-31 - 1993-01-31 # train records 9691 ,# val records 4732
Train period: 1987-01-31 - 1992-01-31 ,val period: 1992-01-31 - 1994-01-31 # train records 9885 ,# val records 5202
Train period: 1988-01-31 - 1993-01-31 ,val period: 1993-01-31 - 1995-01-

MemoryError: Unable to allocate 10.3 TiB for an array with shape (1190354, 1190354) and data type float64

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()
X = df_bottom[features]
y = df_bottom[['risk_premium']]

#Empty containers to save results from each window

predictions_bottom = []
y_val_list_bottom =[]
dates_bottom = []
dic_r2_all_bottom = {}


for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    huber = HuberRegressor(max_iter=1000)

    # fitting the huber model
    huber.fit(X_train, y_train)

    # predicting on the training set and assiging to the training list
    preds_train = huber.predict(X_train)

    preds_val = huber.predict(X_val) 
    predictions_bottom.append(preds_val)
    dates_bottom.append(y_val.index)
    y_val_list_bottom.append(y_val)
    
    #Calculate OOS model performance the for current window
    r2_train = 1-np.sum(pow(y_train['risk_premium']-preds_train,2))/np.sum(pow(y_train['risk_premium'],2))
    r2_val = 1-np.sum(pow(y_val['risk_premium']-preds_val,2))/np.sum(pow(y_val['risk_premium'],2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all_bottom["r2." + str(y_val['risk_premium'].index)] = r2_val
    

    print(f'R2  training set {r2_train}')
    print(f'R2  validation set {r2_val}')
    print('---')

predictions_all_full_bottom = np.concatenate(predictions_bottom, axis=0)
y_test_list_all_full_bottom = np.concatenate(y_val_list_bottom, axis=0)
dates_all_full_bottom = np.concatenate(dates_bottom, axis=0)


R2BOTTOM = r2_score(y_test_list_all_full_top, predictions_all_full_top)
print("R2OOS Linear Regression: ", R2BOTTOM)

In [ ]:
chart = np.array([[R2FULL],
                  [R2TOP],
                  [R2BOTTOM]])

huber = pd.DataFrame(chart, columns=['Huber Regression'],
                     index=['Full Sample', 'Large Firms', 'Small Firms'])

huber

In [ ]:
huber.to_csv(r'huber_model.csv', index=False)

In [ ]:
yhat = predictions_all_full.tolist()
y_true = y_test_list_all_full.tolist()
i = dates_all_full.tolist()

results = pd.DataFrame(
    {'identifier': i,
     'yhat': yhat,
     'y_true': y_true
    })


results["identifier"]= results["identifier"].astype("str")
results["date"] = results["identifier"].str[12:22]
results["id"] = results["identifier"].str[35:40]
results.drop(["identifier"],axis = 1, inplace=True)
results['date'] = pd.to_datetime(results['date'], format='%Y-%m-%d')
results['MonthYear'] = results['date'].dt.to_period('M')
results = results.sort_values(by = ['date', 'id'], ascending = True)
results = results.set_index(['MonthYear','id'])
results.head()


yhat                y_true       date
MonthYear id                                               
1985-01   10137  -8.950644  [-6.745699882507324] 1985-01-31
          10145 -10.684441   [4.993500232696533] 1985-01-31
          10161 -11.244037   [8.103799819946289] 1985-01-31
          10225  -9.830758  [-8.828200340270996] 1985-01-31
          10233  -9.638309  [-6.663899898529053] 1985-01-31

In [32]:
# results['yhat'] = results['yhat'].apply(lambda x: x[0])
results['y_true'] = results['y_true'].apply(lambda x: x[0])

In [33]:
data = df[['mvel12', 'macro_tbl', 'macro_svar']].copy()
data.reset_index(inplace=True)
data['permno2'] = data['permno2'].astype('str')
data['MonthYear'] = data['DATE2'].dt.to_period('M')
data.drop('DATE2', axis=1, inplace=True)
data.rename(columns={'permno2': 'id'}, inplace=True)
data.rename(columns={'mvel12': 'market_cap'}, inplace=True)
data.rename(columns={'macro_tbl': 'risk_free_rate'}, inplace=True)
data = data.set_index(['MonthYear','id'])


In [34]:
bigdata = pd.merge(results, data,left_index=True, right_index=True)
bigdata.reset_index(inplace=True)
bigdata

,MonthYear,id,yhat,y_true,date,market_cap,risk_free_rate,macro_svar
0,1985-01,10137,-8.950644,-6.7457,1985-01-31,-0.412073,0.002625,0.002625
1,1985-01,10145,-10.684441,4.9935,1985-01-31,0.370079,0.002625,0.002625
2,1985-01,10161,-11.244037,8.1038,1985-01-31,-0.769029,0.002625,0.002625
3,1985-01,10225,-9.830758,-8.8282,1985-01-31,0.543307,0.002625,0.002625
4,1985-01,10233,-9.638309,-6.6639,1985-01-31,-0.049869,0.002625,0.002625
...,...,...,...,...,...,...,...,...
1190349,2021-01,93393,-0.503884,-5.4710,2021-01-29,-0.964940,0.000398,0.000398
1190350,2021-01,93419,-0.547618,-2.4747,2021-01-29,-0.098805,0.000398,0.000398
1190351,2021-01,93423,-1.648108,0.2333,2021-01-29,-0.258167,0.000398,0.000398
1190352,2021-01,93427,0.378164,1.6799,2021-01-29,-0.267729,0.000398,0.000398


In [35]:
bigdata['returns'] = bigdata['y_true'] + bigdata['risk_free_rate']
bigdata

,MonthYear,id,yhat,y_true,date,market_cap,risk_free_rate,macro_svar,returns
0,1985-01,10137,-8.950644,-6.7457,1985-01-31,-0.412073,0.002625,0.002625,-6.743075
1,1985-01,10145,-10.684441,4.9935,1985-01-31,0.370079,0.002625,0.002625,4.996125
2,1985-01,10161,-11.244037,8.1038,1985-01-31,-0.769029,0.002625,0.002625,8.106424
3,1985-01,10225,-9.830758,-8.8282,1985-01-31,0.543307,0.002625,0.002625,-8.825576
4,1985-01,10233,-9.638309,-6.6639,1985-01-31,-0.049869,0.002625,0.002625,-6.661275
...,...,...,...,...,...,...,...,...,...
1190349,2021-01,93393,-0.503884,-5.4710,2021-01-29,-0.964940,0.000398,0.000398,-5.470602
1190350,2021-01,93419,-0.547618,-2.4747,2021-01-29,-0.098805,0.000398,0.000398,-2.474302
1190351,2021-01,93423,-1.648108,0.2333,2021-01-29,-0.258167,0.000398,0.000398,0.233698
1190352,2021-01,93427,0.378164,1.6799,2021-01-29,-0.267729,0.000398,0.000398,1.680298


In [36]:
bigdata['MonthYear1'] = bigdata['MonthYear'].copy()
bigdata['MonthYear'] = bigdata['MonthYear'].astype('int64')
bigdata['NumMonth'] = bigdata['MonthYear'] - 251
bigdata['NumMonth'].unique()

array([-71, -70, -69, -68, -67, -66, -65, -64, -63, -62, -61, -60, -59,
       -58, -57, -56, -55, -54, -53, -52, -51, -50, -49, -48, -47, -46,
       -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35, -34, -33,
       -32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22, -21, -20,
       -19, -18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,
        -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,
         7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,
        20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 10

In [37]:

for i in bigdata['NumMonth'].unique():
    globals()['df_' + str(i)] = bigdata[bigdata['NumMonth'] == i]

In [38]:

for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
    
for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["DecileRank"]=pd.qcut(globals()['df_' + str(i)]['rank'].values, 10, labels = False)

#Drop normal rank, retain only decile ranks 
for i in bigdata["NumMonth"].unique():
     globals()['df_' + str(i)].drop('rank', axis=1, inplace=True)

In [39]:

for i in bigdata["NumMonth"].unique():
    for j,g in globals()['df_' + str(i)].groupby('DecileRank'):
        globals()['df_' + str(i)+ "_" + str(j)] =  g

In [40]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)] = pd.concat([globals()['df_1_'+ str(j)], globals()['df_2_'+ str(j)]], axis=0)
    
# Generate 10 Dataframes for the 10 Decile portfolios 0-9: rank_9: top portfolio, rank_0: bottom portfolio
for i in np.arange(2,348,1):
    for j in np.arange(0,10,1):
        globals()['rank_' + str(j)] = pd.concat([globals()['rank_' + str(j)], globals()['df_' + str(i+1)+ "_" + str(j)]], axis = 0)

In [41]:

rank_9

,MonthYear,id,yhat,y_true,date,market_cap,risk_free_rate,macro_svar,returns,MonthYear1,NumMonth,DecileRank
69652,252,10078,-3.958783,26.5585,1991-01-31,-0.108108,0.001931,0.001931,26.560431,1991-01,1,9
69653,252,10078,-2.765651,26.5585,1991-01-31,-0.108108,0.001931,0.001931,26.560431,1991-01,1,9
69655,252,10104,-3.877466,8.0957,1991-01-31,-0.934363,0.001931,0.001931,8.097631,1991-01,1,9
69657,252,10107,-4.299654,24.2087,1991-01-31,0.787645,0.001931,0.001931,24.210631,1991-01,1,9
69670,252,10401,-4.347166,2.1088,1991-01-31,0.972973,0.001931,0.001931,2.110730,1991-01,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...
1161138,599,93246,1.092536,0.6018,2019-12-31,0.245085,0.000437,0.000437,0.602237,2019-12,348,9
1161146,599,93268,12.521050,-12.7764,2019-12-31,-0.852337,0.000437,0.000437,-12.775963,2019-12,348,9
1161152,599,93296,1.422890,3.1023,2019-12-31,0.370031,0.000437,0.000437,3.102737,2019-12,348,9
1161162,599,93330,1.384020,-3.9707,2019-12-31,0.188292,0.000437,0.000437,-3.970263,2019-12,348,9


In [42]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]["eq_weights"] = 1/globals()['rank_' + str(j)].groupby('MonthYear')["id"].transform('size')

In [43]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_stock_ew'] = globals()['rank_' + str(j)]["y_true"]*globals()['rank_' + str(j)]["eq_weights"]

In [44]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_stock_ew'] = globals()['rank_' + str(j)]["returns"]*globals()['rank_' + str(j)]["eq_weights"]

In [45]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["excess_return_stock_ew"].transform('sum')

In [46]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["return_stock_ew"].transform('sum')

In [47]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_stock_ew'] = globals()['rank_' + str(j)]["yhat"]*globals()['rank_' + str(j)]["eq_weights"]

In [48]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["pred_excess_return_stock_ew"].transform('sum')

In [49]:
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)] = globals()['rank_' + str(j)][["MonthYear1", "DecileRank",
                                                                      "excess_return_portfolio_ew",
                                                                      "pred_excess_return_portfolio_ew",
                                                                      "return_portfolio_ew"]]

In [50]:
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].drop_duplicates()
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].set_index("MonthYear1")

In [51]:
for j in np.arange(0,10,1):
    #Time-series average of realized excess returns
    globals()["ew_mean_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()
    #Time-series average of predicted excess returns
    globals()["ew_mean_pred_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["pred_excess_return_portfolio_ew"].mean()
    #Standard deviation of realized excess returns
    globals()["std_ew_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].std()
    #Annualized sharpe ratio of realized excess returns
    globals()["sharpe_ew_rank_" +  str(j)]= (globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()/globals()['montly_rank_' + str(j)]["return_portfolio_ew"].std())* np.sqrt(12)

In [52]:

# For the zero-net-investment long-short portfolio the top (long) and bottom(short) decile portfolios are needed

long_monthly = rank_9[["NumMonth",
                       "MonthYear1",
                       "DecileRank", 
                       "excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_ew",
                       "return_portfolio_ew",
                        ]].drop_duplicates()

short_monthly = rank_0[["NumMonth",
                        "MonthYear1",
                        "DecileRank",
                        "excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_ew",
                       "return_portfolio_ew",
                        ]].drop_duplicates()

# Create a column, indication the stategy 
long_monthly["Strategy"]= "long"
short_monthly["Strategy"]= "short"

# Merge to get the zero net investment portfolio
zeronet_monthly= pd.concat([long_monthly, short_monthly])
zeronet_monthly = zeronet_monthly.sort_values(by = ['NumMonth',"Strategy"])

#Create two new columns containing the exess return of the portfolio and initially set the values to zero.
zeronet_monthly["excess_return_zeronet_ew"] =0


In [53]:
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 3]-zeronet_monthly.iloc[i+1, 3]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 3]-zeronet_monthly.iloc[i, 3]
        


zeronet_monthly["pred_excess_return_zeronet_ew"] =0

for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 4]-zeronet_monthly.iloc[i+1, 4]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 4]-zeronet_monthly.iloc[i, 4]


zeronet_monthly["return_zeronet_ew"] =0

for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 5]-zeronet_monthly.iloc[i+1, 5]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 5]-zeronet_monthly.iloc[i, 5]

In [54]:
#Only the measures at portfolio level are needed
zeronet_monthly = zeronet_monthly[['NumMonth', 
                                   'MonthYear1', 
                                   'excess_return_zeronet_ew',
                                   'pred_excess_return_zeronet_ew',
                                   'return_zeronet_ew',
                                   ]].drop_duplicates()

zeronet_monthly

,NumMonth,MonthYear1,excess_return_zeronet_ew,pred_excess_return_zeronet_ew,return_zeronet_ew
69652,1,1991-01,3.838476,2.535471,3.836982
69710,1,1991-01,-1.386468,2.833107,-1.386468
70688,2,1991-02,-0.201396,-2.024136,-0.201307
70692,2,1991-02,-1.753248,2.825239,-1.753248
71774,3,1991-03,7.145049,-2.343555,7.145139
...,...,...,...,...,...
1147601,346,2019-10,1.518548,2.856472,1.518548
1152070,347,2019-11,-5.242852,-2.836011,-5.242848
1152171,347,2019-11,2.531833,2.803048,2.531833
1156610,348,2019-12,0.976897,-2.790520,0.976901


In [55]:
#Calculate zero-net portfolio performance measures                                            
#Time-series average of realized excess returns                                             
ew_mean_return_zeronet= zeronet_monthly["excess_return_zeronet_ew"].mean()
#Time-series average of predicted excess returns
ew_mean_pred_return_zeronet = zeronet_monthly["pred_excess_return_zeronet_ew"].mean()
#Standard deviation of realized excess returns
std_ew_zeronet = zeronet_monthly["excess_return_zeronet_ew"].std()
#Annualized sharpe ratio of realized excess returns
sharpe_ew_zeronet = (zeronet_monthly["excess_return_zeronet_ew"].mean()/zeronet_monthly["return_zeronet_ew"].std())* np.sqrt(12)

In [56]:
zeronet_monthly['excess_return_zeronet_ew']

69652      3.838476
69710     -1.386468
70688     -0.201396
70692     -1.753248
71774      7.145049
             ...   
1147601    1.518548
1152070   -5.242852
1152171    2.531833
1156610    0.976897
1156709   -3.798404
Name: excess_return_zeronet_ew, Length: 696, dtype: float64

In [57]:
chart_np = np.array([[ew_mean_pred_return_rank_0, ew_mean_return_rank_0, std_ew_rank_0, sharpe_ew_rank_0],
                     [ew_mean_pred_return_rank_1, ew_mean_return_rank_1, std_ew_rank_1, sharpe_ew_rank_1],
                     [ew_mean_pred_return_rank_2, ew_mean_return_rank_2, std_ew_rank_2, sharpe_ew_rank_2],
                     [ew_mean_pred_return_rank_3, ew_mean_return_rank_3, std_ew_rank_3, sharpe_ew_rank_3],
                     [ew_mean_pred_return_rank_4, ew_mean_return_rank_4, std_ew_rank_4, sharpe_ew_rank_4],
                     [ew_mean_pred_return_rank_5, ew_mean_return_rank_5, std_ew_rank_5, sharpe_ew_rank_5],
                     [ew_mean_pred_return_rank_6, ew_mean_return_rank_6, std_ew_rank_6, sharpe_ew_rank_6],
                     [ew_mean_pred_return_rank_7, ew_mean_return_rank_7, std_ew_rank_7, sharpe_ew_rank_7],
                     [ew_mean_pred_return_rank_8, ew_mean_return_rank_8, std_ew_rank_8, sharpe_ew_rank_8],
                     [ew_mean_pred_return_rank_9, ew_mean_return_rank_9, std_ew_rank_9, sharpe_ew_rank_9],
                     [ew_mean_pred_return_zeronet, ew_mean_return_zeronet, std_ew_zeronet, sharpe_ew_zeronet]])

ew_df = pd.DataFrame(chart_np, columns=['Pred', 'Real', 'Std', 'Sharpe'],
                              index=['Low (L)', '2', '3', '4', '5','6','7','8',"9",'High (H)', "H-L"])

ew_df['Pred'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Pred']], index = ew_df.index)
ew_df['Real'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Real']], index = ew_df.index)
ew_df['Std'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Std']], index = ew_df.index)
ew_df['Sharpe'] = pd.Series([("%.2f" % round(val, 2)) for val in ew_df['Sharpe']], index = ew_df.index)
ew_df

,Pred,Real,Std,Sharpe
Low (L),-4.97%,-2.15%,6.01%,-1.24
2,-4.23%,-1.85%,5.05%,-1.27
3,-3.88%,-1.80%,4.83%,-1.29
4,-3.59%,-1.75%,4.74%,-1.28
5,-3.29%,-1.71%,4.69%,-1.26
6,-2.78%,-1.79%,4.81%,-1.29
7,-2.48%,-1.70%,4.77%,-1.24
8,-2.20%,-1.68%,4.74%,-1.23
9,-1.87%,-1.66%,4.83%,-1.19
High (H),-1.28%,-1.62%,5.26%,-1.07
